In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import os
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score
import json
import numpy as np
from tqdm import tqdm
import xgboost as xgb

In [2]:
## remember to rajouter AttackComplexity
labels = ['attackVector', 'availabilityImpact', 'confidentialityImpact',
          'integrityImpact', 'privilegesRequired', 'scope', 'userInteraction']

In [4]:
scores = {}
ngrams = {
    "bi-trigrams": (2,3)
}
for key,value in tqdm(ngrams.items()):
    for label in tqdm(labels):
        with open(f'data/{label}/label_mapping.txt') as f:
            content = f.read().replace("'",'"')
            class_names = list(json.loads(content).keys())

        train = pd.read_csv(f'data/{label}/train.csv')
        test = pd.read_csv(f'data/{label}/test.csv')

        x_train, y_train = train['text'], train['labels']
        x_valid, y_valid   = test['text'], test['labels']

        vec = TfidfVectorizer(decode_error='ignore',stop_words='english', max_df=0.8, max_features=10000, ngram_range=value)

        x_train_vec = vec.fit_transform(x_train).todense()
        x_train_vec = pd.DataFrame(x_train_vec, columns=vec.get_feature_names_out()).astype(np.float16)

        x_valid_vec = vec.transform(x_valid).todense()
        x_valid_vec = pd.DataFrame(x_valid_vec, columns=vec.get_feature_names_out()).astype(np.float16)


        model = XGBClassifier(learning_rate=0.5, random_state=1, tree_method='gpu_hist',  
                              gpu_id=0, predictor="gpu_predictor", n_estimators=300)

        model.fit(x_train_vec, y_train)

        preds = model.predict(x_valid_vec)
        
        scores.update({
            label: {
                "Accuracy": round(accuracy_score(preds, y_valid),3),
                "F1": round(f1_score(preds, y_valid,average="weighted"),3)
            }
        })

        model.save_model(f"models/xgb_{key}/{label}.json")
        print (label, scores[label])

    with open(f"models/xgb_{key}/scores.txt",'w') as f:
        f.write(json.dumps(scores))

 14%|███████████▊                                                                       | 1/7 [02:30<15:03, 150.65s/it]

attackVector {'Accuracy': 0.904, 'F1': 0.909}



 29%|███████████████████████▋                                                           | 2/7 [04:36<11:20, 136.06s/it]

availabilityImpact {'Accuracy': 0.872, 'F1': 0.875}



 43%|███████████████████████████████████▌                                               | 3/7 [06:44<08:50, 132.58s/it]

...

privilegesRequired {'Accuracy': 0.822, 'F1': 0.83}



 86%|███████████████████████████████████████████████████████████████████████▏           | 6/7 [12:07<01:48, 108.42s/it]

scope {'Accuracy': 0.957, 'F1': 0.959}



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [13:15<00:00, 795.44s/it]

userInteraction {'Accuracy': 0.92, 'F1': 0.922}


In [9]:
scores = {}

for label in tqdm(labels):
    train = pd.read_csv(f'data/{label}/train.csv')
    test = pd.read_csv(f'data/{label}/test.csv')

    x_train, y_train = train['text'], train['labels']
    x_valid, y_valid   = test['text'], test['labels']

    vec = TfidfVectorizer(decode_error='ignore',stop_words='english', max_df=0.8, max_features=15000, ngram_range=(1,3))

    x_train = vec.fit_transform(x_train).todense()
    x_train = pd.DataFrame(x_train, columns=vec.get_feature_names_out()).astype(np.float16)

    x_valid = vec.transform(x_valid).todense()
    x_valid = pd.DataFrame(x_valid, columns=vec.get_feature_names_out()).astype(np.float16)

    model = XGBClassifier(learning_rate=0.1, random_state=1, tree_method='gpu_hist',  
                          gpu_id=0, predictor="gpu_predictor", n_estimators=300)

    model.fit(x_train, y_train)
    
    preds = model.predict(x_valid)

    scores.update({
        label: {
            "Accuracy": round(accuracy_score(preds, y_valid),3),
            "F1": round(f1_score(preds, y_valid,average="weighted"),3)
        }
    })

    model.save_model(f"models/xgb_allgrams/{label}.json")
    print (label, scores[label])

with open(f"models/xgb_allgrams/scores.txt",'w') as f:
    f.write(json.dumps(scores))

  0%|                                                                                            | 0/7 [00:31<?, ?it/s]


XGBoostError: [02:26:05] C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/tree/updater_gpu_hist.cu:798: Exception in gpu_hist: [02:26:05] c:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1\xgboost\xgboost-ci-windows\src\data\../common/device_helpers.cuh:431: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory
- Free memory: 0
- Requested memory: 655110000

